<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/title.png' width=500/></center>

# 4.데이터프레임 집계

- 상세 데이터가 아닌 집계된 데이터에 대한 분석을 자주 요구하니 익숙해져야 할 내용입니다.
- sum(), mean(), max(), min(), count() 메서드를 사용해 지정한 열 또는 열들을 기준으로 집계합니다.
- 평균을 구하는 메서드가 avg()가 아닌 mean() 임을 주의하기 바랍니다.

In [ ]:
# 라이브러리 불러오기
import pandas as pd

In [ ]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/tips.csv'
tip = pd.read_csv(path)

# 확인
tip

- 우선 집계에 대하여 다음과 같이 정리해 볼 수 있습니다.

<img src='https://raw.githubusercontent.com/jangrae/img/master/groupby.png' width=500 align="left"/>

- **연속값**, **범주값**이라는 용어에 익숙해 지길 바랍니다.
- 대부분은 범주값을 기준으로 연속값의 합, 평균 등을 집계합니다.
- 가끔은 범주값의 개수나 최빈값을 집계하기도 합니다.
- 집계 결과에 대한 시각화를 통해 더 나은 통찰력을 얻게 합니다.

## 4.1.합 구하기

### 4.1.1.열 하나 집계

- 우선 특정 열의 값 합은 다음과 같이 구할 수 있습니다.

In [ ]:
# total_bill 합계
tip['total_bill'].sum()

In [ ]:
# total_bill, tip 각각의 합계
tip[['total_bill', 'tip']].sum()

**1) 집계하기**

- 만일 day 별로 합을 구하고자 한다면 다음과 같이 합니다.
- 아래 결과 값 네 개를 더하면 전체 합이 됩니다.
- **as_index=True**를 설정(기본값)하면 집계 기준이 되는 열이 인덱스 열이 됩니다.
- 집계 결과가 tip 열만 가지니 시리즈가 됩니다.

In [ ]:
# day별 tip 합계 --> 시리즈
tip.groupby('day', as_index=True)['tip'].sum()

- **[['tip']].sum()** 과 같이 하면 열이 여럿이라는 의미여서 결과가 **데이터프레임**이 됩니다.

In [ ]:
# day별 tip 합계 --> 데이터프레임
tip.groupby('day', as_index=True)[['tip']].sum()

- **as_index=False**를 설정하면 행 번호를 기반으로 한 정수 값이 인덱스로 설정됩니다.

In [ ]:
# day별 tip 합계 --> 데이터프레임
tip.groupby('day', as_index=False)[['tip']].sum()

**2) 데이터프레임으로 선언**

- 집계 결과를 새로운 데이터프레임으로 선언하여 사용하는 경우가 많습니다.
- 집계된 결과를 반복해서 사용하거나, 분석 대상이 되는 경우 데이터프레임으로 선언함이 유익합니다.

In [ ]:
# day별 tip 합계 조회
tip_sum = tip.groupby('day', as_index=False)[['tip']].sum()

# 확인
tip_sum

**3) 집계 결과 시각화**

- 위 과정으로 집계한 결과를 막대그래프로 간단히 시각화 해봅니다.
- matplotlib 패키지의 pyplot을 plt 별칭을 주어 불러와 시각화에 사용합니다.

In [ ]:
# 라이브러리 불러오기
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

# day 별 tip 비교 시각화
plt.rc('axes', axisbelow=True)
plt.figure(figsize=(8, 5))
plt.bar(x=tip_sum['day'], height=tip_sum['tip'])
plt.title('Tips by Day', size=15, pad=20, fontweight='bold')
plt.xlabel('Day')
plt.ylabel('Tips')
plt.grid(axis='y')   # x, y, both
plt.show()

- 막대그래프는 기본적으로 x축 값을 기준(위의 경우 요일명)으로 정렬이 됩니다.
- 임의의 순서로 정렬하려면 추가적인 코딩이 필요합니다.

In [ ]:
# day 별 tip 비교 시각화(임의 순서 지정)
tip_sum['orders'] = tip_sum['day']
tip_sum['orders'] = tip_sum['orders'].map({'Thur':0, 'Fri':1, 'Sat':2, 'Sun':3})
plt.figure(figsize=(8, 5))
plt.bar(tip_sum['orders'], tip_sum['tip'])
plt.xticks([0, 1, 2, 3], ['Thur', 'Fri', 'Sat', 'Sun'])
plt.show()

- plt.bar() 대신 plt.barh() 함수를 사용해 가로 막대그래프를 표시할 수 있습니다.

In [ ]:
# day 별 tip 비교 시각화
plt.figure(figsize=(8, 5))
# plt.barh(y=tip_sum['day'], width=tip_sum['tip'], color='tab:blue')
# plt.barh(y=tip_sum['day'], width=tip_sum['tip'], color='tab:orange')
# plt.barh(y=tip_sum['day'], width=tip_sum['tip'], color='tab:green')
plt.barh(y=tip_sum['day'], width=tip_sum['tip'], color='g')
plt.show()

**4) 참고: 선 그래프**

- 연속형 값의 변화 추이를 볼 때 선 그래프가 유용합니다.

In [ ]:
# tip 변경 추이 시각화
plt.figure(figsize=(8, 5))
plt.plot(tip['tip'])
plt.show()

In [ ]:
# tip, total_bill 변경 추이 시각화
plt.figure(figsize=(8, 5))
plt.plot(tip[['tip', 'total_bill']])
plt.legend(['tip', 'total_bill'], loc='upper left')   # loc: upper left, lower right, upper center...
plt.show()

**5) 참고: 히스토그램**

- 연속형 값의 분포를 볼 때는 히스토그램이 유용합니다.

In [ ]:
# tip 분포 시각화
plt.figure(figsize=(8, 5))
plt.hist(tip['tip'], bins=20, alpha=.7, edgecolor='w')
plt.show()

In [ ]:
# total_bill 분포 시각화
total_bill_mean = tip['total_bill'].median()
plt.figure(figsize=(8, 5))
plt.hist(tip['total_bill'], bins=20, alpha=.7, edgecolor='w')
plt.axvline(total_bill_mean, color='r')
plt.show()

In [ ]:
# total_bill 분포 시각화
total_bill_mean = tip['total_bill'].median()
plt.figure(figsize=(8, 5))
plt.hist(tip['total_bill'], bins=20, alpha=.7, edgecolor='w')
plt.axvline(total_bill_mean, color='r')
#plt.text(total_bill_mean + 2, 35, 'mediam', size=8, color='r')
plt.annotate('median',
             xy=(total_bill_mean, 35),
             xytext=(total_bill_mean + 5, 35),
             arrowprops=dict(arrowstyle='-|>', color='r'),
             size=8,
             color='r')
plt.show()

**6) 참고: 박스그래프**

- 연속형 값의 분포를 볼 때는 히스토그램이 유용합니다.

In [ ]:
# tip 분포 시각화
plt.figure(figsize=(8, 5))
plt.boxplot(tip['tip'], vert=False)
plt.show()

- 히스토그램과 박스그래프를 동시에 표현해 데이터 분포를 확인할 수 있습니다.

In [ ]:
# tip 분포 시각화
plt.figure(figsize=(8, 5))
plt.subplot(2, 1, 1)
plt.hist(tip['tip'], bins=20, alpha=.7, edgecolor='w')
plt.subplot(2, 1, 2)
plt.boxplot(tip['tip'], vert=False)
plt.show()

**7) 참고: 산점도**

- 열 간의 상관관계를 확인할 때는 산점도(Scatter)가 유용합니다.

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(tip['total_bill'], tip['tip'])
plt.xlabel('total_bill')
plt.ylabel('tip')
plt.show()

### 4.1.2.여러 열 집계

- 여러 열에 대한 집계를 같이 할 수 있습니다.
- **[ ['total_bill', 'tip'] ].sum()** 형태와 같이 집계 대상 열을 리스트로 지정합니다.

In [ ]:
# day별 total_bill, tip 합계 조회
tip_sum = tip.groupby('day', as_index=False)[['total_bill', 'tip']].sum()

# 확인
tip_sum

- sum() 메서드 앞에 아무 열도 지정하지 않으면 **기준열 이외의 모든 열에 대한 집계**가 수행됩니다.

In [ ]:
# day별 나머지 열들 합계 조회
tip_sum = tip.groupby('day', as_index=False).sum(numeric_only=True)

# 확인
tip_sum

- **by=['day', 'smoker']** 과 같이 집계 기준 열을 여럿 설정할 수도 있습니다.

In [ ]:
# day + smoker별 나머지 열들 합계 조회
tip_sum = tip.groupby(['day', 'smoker'], as_index=False).sum(numeric_only=True)

# 확인
tip_sum

## 4.2.평균, 최댓값, 최솟값, 개수 구하기

- 위와 동일한 방법으로 평균, 최댓값, 최솟값, 개수를 집계할 수 있습니다.
- sum() 메서드 대신 관련된 메서드만 적절히 지정하면 됩니다.

**1) 평균 구하기**

In [ ]:
# day별 tip 평균
tip_mean = tip.groupby('day', as_index=False)[['tip']].mean(numeric_only=True)

# 확인
tip_mean

In [ ]:
# day + sex 별 나머지 모든 열 평균
tip_mean = tip.groupby(['day', 'sex'], as_index=False).mean(numeric_only=True)

# 확인
tip_mean

**2) 최댓값, 최솟값, 개수 구하기**


In [ ]:
# day별 tip 최댓값
tip_max = tip.groupby('day', as_index=False)[['tip']].max()

# 확인
tip_max

In [ ]:
# day별 tip 최솟값
tip_min = tip.groupby('day', as_index=False)[['tip']].min()

# 확인
tip_min